In [98]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue
import numpy as np

In [99]:
PROBLEM_SIZE = 15
NUM_SETS = 30
SETS = tuple(np.array([random() < 0.25 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)) #Set are created randomly, 25% true 75% false

State = namedtuple('State', ['taken', 'not_taken'])


In [100]:
def check_super_sets():   #I check if there are sets with all True or all False values
    filtered_sets = tuple(filter(any, SETS)) 
    all_true_sets = tuple(filter(all, SETS)) 
    return filtered_sets, all_true_sets 
 
filtered_sets, all_true_sets = check_super_sets() #filtered sets without sets with all false
if(len(all_true_sets) != 0): 
    print("Problem solvable with only one set") 
NUM_FSETS = len(filtered_sets)  #number of filtered sets without sets with all false
print(NUM_FSETS) 
#print(filtered_sets)

29


In [101]:
def covered(state):
    return reduce(
        np.logical_or,
        [filtered_sets[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def goal_check(state):
    return np.all(covered(state))

assert goal_check(State(set(range(NUM_FSETS)), set())), "Probelm not solvable"

In [102]:
def h_largest_set(state):
    largest_set_size = max(sum(s) for s in filtered_sets) 
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size) 
    return optimistic_estimate


def h2_largest_uncovered_set(state):
    already_covered = covered(state)  #remove the taken nodes before considering the largest_set_size available!
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in filtered_sets)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3_largest_uncovered_ordered_set(state):   #remove the taken nodes before considering the largest_set_size available and then ordered them descendently!
    already_covered = covered(state)  
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in filtered_sets), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size: 
        taken += 1
    return taken
# With h3 as heuristic function we obtain an improvement of 4 orders of magnitude compared to breadth first

def f(state):
    return len(state.taken) + h3_largest_uncovered_ordered_set(state)

In [103]:
frontier = PriorityQueue()  
state = State(set(), set(range(NUM_FSETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
    
print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

Solved in 20 steps (3 tiles)


In [104]:
current_state


State(taken={17, 3, 20}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28})